# login

from django.contrib.auth import authenticate, login, logout

一般用户登录先是authenticate通过得到一个user_obj对象，然后调用login(request, user_obj). 我们可以猜想一下这个函数做了哪些事？

首先，涉及到用户登录一般会涉及到session和cookie, 用户登录给浏览器设置cookie, 服务器保存session. 那么服务器端保存的session的值是什么呢？存的是{'_auth_user_id': '1', '_auth_user_backend': 'django.contrib.auth.backends.ModelBackend', '_auth_user_hash': '2b759eb824f9031102507260e0d512dc0d33056c'}。 我们应该关注的是'_auth_user_id', session中存了用户id, 通过这个id就能查到用户的其他信息(如用户名，密码等)。django内置的login其实就是把用户id放在session里，本质上还是操作的还是request.session。

In [ ]:


ef login(request, user, backend=None):
    """
    Persist a user id and a backend in the request. This way a user doesn't
    have to reauthenticate on every request. Note that data set during
    the anonymous session is retained when the user logs in.
    """
    session_auth_hash = ''
    if user is None:
        user = request.user
    if hasattr(user, 'get_session_auth_hash'):
        session_auth_hash = user.get_session_auth_hash()
    # SESSION_KEY: _auth_user_id
    if SESSION_KEY in request.session:
        if _get_user_session_key(request) != user.pk or (
                session_auth_hash and
                not constant_time_compare(request.session.get(HASH_SESSION_KEY, ''), session_auth_hash)):
            # To avoid reusing another user's session, create a new, empty
            # session if the existing session corresponds to a different
            # authenticated user.
            request.session.flush()
    else:
        request.session.cycle_key()

    try:
        backend = backend or user.backend
    except AttributeError:
        backends = _get_backends(return_tuples=True)
        if len(backends) == 1:
            _, backend = backends[0]
        else:
            raise ValueError(
                'You have multiple authentication backends configured and '
                'therefore must provide the `backend` argument or set the '
                '`backend` attribute on the user.'
            )
    else:
        if not isinstance(backend, str):
            raise TypeError('backend must be a dotted import path string (got %r).' % backend)

    request.session[SESSION_KEY] = user._meta.pk.value_to_string(user)
    request.session[BACKEND_SESSION_KEY] = backend
    request.session[HASH_SESSION_KEY] = session_auth_hash
    if hasattr(request, 'user'):
        request.user = user
    rotate_token(request)
    user_logged_in.send(sender=user.__class__, request=request, user=user)

logout 用于清除后端数据库session记录，并把给前端的cookie置为空。

In [ ]:
def logout(request):
    """
    Remove the authenticated user's ID from the request and flush their session
    data.
    """
    # Dispatch the signal before the user is logged out so the receivers have a
    # chance to find out *who* logged out.
    user = getattr(request, 'user', None)
    if not getattr(user, 'is_authenticated', True):
        user = None
    user_logged_out.send(sender=user.__class__, request=request, user=user)

    # remember language choice saved to session
    language = request.session.get(LANGUAGE_SESSION_KEY)

    request.session.flush()

    if language is not None:
        request.session[LANGUAGE_SESSION_KEY] = language

    if hasattr(request, 'user'):
        from django.contrib.auth.models import AnonymousUser
        request.user = AnonymousUser()

# request.user

request 之所以有user这个属性，是中间件AuthenticationMiddleware做的妖。可以看看这个中间件是怎么往里面request这个对象里放置user属性的。

from django.contrib.auth.middleware import AuthenticationMiddleware.

request.user有一个is_authenticated方法，因为request.user就是从session里取到的用户或者是没有登录的匿名用户，对于匿名用户，也就是在session表里没有对应记录的用户is_authenticated返回False.通过认证并不意味着用户拥有任何权限，甚至也不检查该用户是否处于激活状态，这只是表明用户成功的通过了认证。 这个方法很重要, 在后台用request.user.is_authenticated()判断用户是否已经登录，如果true则可以向前台展示request.user.name



In [ ]:
def get_user(request):
    if not hasattr(request, '_cached_user'):
        request._cached_user = auth.get_user(request)
    return request._cached_user


class AuthenticationMiddleware(MiddlewareMixin):
    def process_request(self, request):
        assert hasattr(request, 'session'), (
            "The Django authentication middleware requires session middleware "
            "to be installed. Edit your MIDDLEWARE%s setting to insert "
            "'django.contrib.sessions.middleware.SessionMiddleware' before "
            "'django.contrib.auth.middleware.AuthenticationMiddleware'."
        ) % ("_CLASSES" if settings.MIDDLEWARE is None else "")
        request.user = SimpleLazyObject(lambda: get_user(request))
        
def get_user(request):
    """
    Return the user model instance associated with the given request session.
    If no user is retrieved, return an instance of `AnonymousUser`.
    """
    from .models import AnonymousUser
    user = None
    try:
        # 取到session里的用户id
        user_id = _get_user_session_key(request)
        backend_path = request.session[BACKEND_SESSION_KEY]
    except KeyError:
        pass
    else:
        if backend_path in settings.AUTHENTICATION_BACKENDS:
            backend = load_backend(backend_path)
            # 根据id找到直接定位到用户
            user = backend.get_user(user_id)
            ...

    return user or AnonymousUser()

# auth里的方法

* authenticate():提供了用户认证，即验证用户名以及密码是否正确,一般需要username  password两个关键字参数
* login(HttpRequest, user): 该函数接受一个HttpRequest对象，以及一个认证了的User对象. <font color="red">此函数使用django的session框架给某个已认证的用户附加上session id等信息。</font>
* logout(request) 注销用户: 该函数接受一个HttpRequest对象，无返回值。当调用该函数时，当前请求的session信息会全部清除。该用户即使没有登录，使用该函数也不会报错。

## user对象

User 对象属性：username， password（必填项）password用哈希算法保存到数据库

* is_staff: 用户是否拥有网站的管理权限，拥有staff身份的用户可以登录django的admin后台
* is_active: 是否允许用户登录, 设置为False，可以不用删除用户来禁止用户登录
* create_user: 创建用户
* check_password(passwd): 检查密码是否正确
* set_password(passwd): 修改密码

## 装饰器

* login_required([redirect_field_name=REDIRECT_FIELD_NAME, login_url=None]: 配合settings.LOGIN_URL, login_required装饰器不检查user的is_active标志位!
* permission_required(perm[, login_url=None, raise_exception=False])：has_perm() 方法, 权限名称采用如下方法 "<app label>.<permission codename>" (例如 polls.can_vote 表示在 polls 应用下一个模块的权限!

## 给已验证登录的用户添加访问限制
user_passes_test(func[, login_url=None, redirect_field_name=REDIRECT_FIELD_NAME])

```
from django.shortcuts import redirect

def my_view(request):
    if not request.user.email.endswith('@example.com'):
        return redirect('/login/?next=%s' % request.path)
```

```
from django.contrib.auth.decorators import user_passes_test

def email_check(user):
    return user.email.endswith('@example.com')

@user_passes_test(email_check)
def my_view(request):
    ...
```

user_passes_test() 要求一个以User 对象为参数的回调函数，若用户允许访问此视图，返回 True。注意，user_passes_test() 不会自动检查 User 是否为匿名对象。
user_passes_test()接收两个额外的参数：
* login_url: 指定那些没有通过检查的用户要重定向至哪里。若不指定其值，它可能是默认的 settings.LOGIN_URL。
* redirect_field_name: 与login_required()的参数相同。




## User权限相关的2个属性

1. groups: 多对多属性
2. user_permissions: 多对多属性

## Permission

Django的auth系统提供了模型级的权限控制， 即可以检查用户是否对某个数据表拥有增(add), 改(change), 删(delete)权限。auth系统无法提供对象级的权限控制，即检查用户是否对数据表中某条记录拥有增改删的权限。如果需要对象级权限控制可以使用django-guardian。假设在博客系统中有一张article数据表管理博文，auth可以检查某个用户是否拥有对所有博文的管理权限，但无法检查用户对某一篇博文是否拥有管理权限。

### 创建用户权限

```
content_type = ContentType.objects.get_for_model(Blog)
permission = Permission.objects.create(codename='can_publish',
                                       name='Can Publish Posts',
                                       content_type=content_type)
```

### 检查用户权限
user.has_perm方法用于检查用户是否拥有操作某个模型的权限:
```
user.has_perm('blog.add_article')
user.has_perm('blog.change_article')
user.has_perm('blog.delete_article')
```
上述语句检查用户是否拥有blog这个app中article模型的添加权限， 若拥有权限则返回True。

has_perm仅是进行权限检查, 即用户没有权限它也不会阻止程序员执行相关操作。

@permission_required装饰器可以代替has_perm并在用户没有相应权限时重定向到登录页或者抛出异常。

```
# permission_required(perm[, login_url=None, raise_exception=False])

@permission_required('blog.add_article')
def post_article(request):
    pass
```

每个模型默认拥有增(add), 改(change), 删(delete)权限。在django.contrib.auth.models.Permission模型中保存了项目中所有权限。

该模型在数据库中被保存为auth_permission数据表。每条权限拥有id ,name , content_type_id, codename四个字段。

### 管理用户权限
User和Permission通过多对多字段user.user_permissions关联，在数据库中由auth_user_user_permissions数据表维护。

```
#添加权限
user.user_permissions.add(permission)

#删除权限: 
user.user_permissions.delete(permission)

#清空权限: 
user.user_permissions.clear()
```
<font color="red">用户拥有他所在用户组的权限，</font>使用用户组管理权限是一个更方便的方法。Group中包含多对多字段permissions，在数据库中由auth_group_permissions数据表维护。
```
#添加权限: 
group.permissions.add(permission)

#删除权限: 
group.permissions.delete(permission)

#清空权限: 
group.permissions.clear()
```
### 自定义权限

在定义Model时可以使用Meta自定义权限：
```
class Discussion(models.Model):
  ...
  class Meta:
      permissions = (
          ("create_discussion", "Can create a discussion"),
          ("reply_discussion", "Can reply discussion"),
      )
```
判断用户是否拥有自定义权限:

user.has_perm('blog.create_discussion')

<font color="red">对象级权限：
    
https://django-guardian.readthedocs.io/en/stable/userguide/remove.html 
   
   https://www.jianshu.com/p/01126437e8a4</font>

# 单用户登录

思路就是在用户表额外存一个session_key的字段，每次用户登录就判断有没有这个session_key，如果有说明已经登录过了，那么去session表里把这个session_key对应的记录删除。每次用户注销的时候也把这个session_key一并删除。

In [ ]:
from django.shortcuts import redirect
from django.views.decorators.cache import never_cache
from django.views.decorators.csrf import csrf_exempt
from django.contrib.auth import login as auth_login, authenticate
from app01.models import UCenter # 用户信息表
 
 
@csrf_exempt
@never_cache
def login(request):
  if request.user.is_authenticated():
    return redirect('/index/')
  else:
    if request.method == "POST":
      username = request.POST.get("username")
      password = request.POST.get("password")
      authenticated_user = authenticate(username=username, password=password)
      if authenticated_user:
         
        # 单用户登录
        user_obj = UCenter.objects.filter(userid=authenticated_user) # 找到登录的user对象
        is_session_key = user_obj.first().session_key # 获取登录对象的session_key
        if is_session_key: # 用户已登录
          request.session.delete(is_session_key) # 删除登录前面登录用户的session_key
        auth_login(request, authenticated_user) # 用户信息存入session
        user_obj.update(session_key=request.session.session_key) # 更新新登录user的session_key
        return redirect('/index/')
      else:
        return redirect('/accounts/login/')